# UniqueAuthorsStatistic 开发笔记

本 notebook 演示如何在快速上手框架中开发、调试并导出一个自定义统计卡片。

**English:** This notebook shows how to develop, debug, and export a custom statistic card within the quickstart framework.  
**日本語:** このノートブックでは、クイックスタートフレームワークでカスタム統計カードを開発・デバッグ・エクスポートする方法を説明します。

## 数据准备

运行下方单元以加载示例运行数据。如果你的 `artifacts` 目录下有多个项目，可以自行修改 `repo_name` 和 `run_name` 变量以选中想要调试的执行结果。

**English:** Run the cell below to load sample run data. If multiple projects exist under your `artifacts` directory, adjust the `repo_name` and `run_name` variables to choose the execution you want to debug.  
**日本語:** 下のセルを実行してサンプルの実行データを読み込みます。`artifacts` ディレクトリに複数のプロジェクトがある場合は、調整したい実行を選べるように `repo_name` と `run_name` の変数を変更してください。

In [ ]:
import sys
from pathlib import Path

NOTEBOOKS_DIR = Path.cwd().resolve().parent
if str(NOTEBOOKS_DIR) not in sys.path:
    sys.path.insert(0, str(NOTEBOOKS_DIR))

from quickstart_dashboard import RunDataLoader

loader = RunDataLoader(base_dir="../artifacts")
repos = loader.list_repos()

if not repos:
    print("⚠️ 未找到任何项目，请确认 ../artifacts 目录存在分析结果。")
    repo_name = None
    run_name = None
    sample_run = None
    sample_history = None
else:
    repo_name = repos[0]
    print(f"使用示例项目: {repo_name}")
    runs = loader.list_runs(repo_name)
    if not runs:
        print("⚠️ 项目下暂未找到运行记录。")
        run_name = None
        sample_run = None
        sample_history = None
    else:
        run_name = runs[0]
        print(f"使用示例运行: {run_name}")
        sample_run = loader.load_run(repo_name, run_name)
        sample_history = loader.load_history(repo_name, limit=20)


## 定义统计卡片

在下方代码单元中实现统计逻辑。运行单元后，类会出现在当前 Python 会话中，供后续单元调试或导出使用。

**English:** Implement the statistic logic in the code cell below. After running the cell, the class becomes available in the current Python session for further previewing or exporting.  
**日本語:** 以下のコードセルで統計ロジックを実装します。セルを実行すると、この Python セッションでクラスを利用でき、以降のセルでプレビューやエクスポートに使えます。

In [ ]:
from typing import Optional

import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

from quickstart_dashboard import BaseStatistic, RunData, RunHistory, _stat_card


class UniqueAuthorsStatistic(BaseStatistic):
    """统计 diff 结果中出现的唯一作者数量。"""

    def __init__(self) -> None:
        self.name = "唯一作者数"
        self.description = "根据 diff_results.csv 的 author 列计算"

    def _extract(self, run: RunData) -> Optional[int]:
        df = run.dataframes.get("diff_results_df") if run else None
        if df is None or "author" not in df.columns:
            return None
        return int(pd.Series(df["author"]).dropna().nunique())

    def render_single(self, run: RunData):
        value = self._extract(run)
        display_value = value if value is not None else "N/A"
        return _stat_card(self.name, display_value, self.description)

    def render_trend(self, history: RunHistory):
        records = []
        for run in history.runs if history else []:
            value = self._extract(run)
            if value is not None:
                records.append({"label": run.display_label, "value": value})

        if not records:
            return widgets.HTML(value="<div style='color:#666;'>暂无唯一作者统计数据。</div>")

        df = pd.DataFrame(records)
        output = widgets.Output()
        with output:
            fig, ax = plt.subplots(figsize=(6, 3))
            ax.plot(df["label"], df["value"], marker="o")
            ax.set_title(self.name)
            ax.set_xlabel("Run")
            ax.set_ylabel("Authors")
            ax.grid(True, linestyle="--", alpha=0.3)
            plt.xticks(rotation=30, ha="right")
            plt.tight_layout()
            from IPython.display import display
            display(fig)
            plt.close(fig)

        header = widgets.HTML(
            value=f"<b>{self.name}</b><div style='color:#666;font-size:11px;'>{self.description}</div>"
        )
        return widgets.VBox([header, output])


## 调试与预览

执行以下单元可以在 notebook 中直接查看卡片效果，便于在整合至仪表盘前快速验证。

**English:** Run the following cells to preview the card inside the notebook, making it easy to verify before integrating it into the dashboard.  
**日本語:** 次のセルを実行すると、ノートブック上でカードの表示を確認でき、ダッシュボードに組み込む前に素早く検証できます。

In [ ]:
if sample_run is None:
    print("⚠️ 没有可用的运行数据，无法预览单次统计卡片。")
else:
    stat = UniqueAuthorsStatistic()
    stat.render_single(sample_run)


In [ ]:
if sample_history is None or not sample_history.runs:
    print("⚠️ 没有足够的历史数据，无法绘制趋势图。")
else:
    stat = UniqueAuthorsStatistic()
    stat.render_trend(sample_history)


## 导出到仪表盘

确认逻辑后，可以在 `quickstart_dashboard.ipynb` 中通过 `load_statistic_from_notebook` 将此类加载并注册到仪表盘。

**English:** After confirming the logic, load and register this class in `quickstart_dashboard.ipynb` via `load_statistic_from_notebook`.  
**日本語:** ロジックを確認したら、`quickstart_dashboard.ipynb` で `load_statistic_from_notebook` を使ってこのクラスを読み込み、ダッシュボードに登録してください。